# GUI Review Truth

Review all extracted frames for a locality and quickly tag whether it should be hit or miss

## Configuration

In [1]:
#subdir = 'dashcam_tour_frankston'
subdir = 'dashcam_tour_mount_eliza'

detections_dir = 'detections_V4'

## Code

In [2]:
import os
import sys
import ipywidgets as widgets
import pandas as pd

from pathlib import Path
from shutil import copyfile

module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)

In [3]:
detections_dir     = os.path.join(os.pardir, 'data_sources', subdir, detections_dir)
hits_dir           = os.path.join(detections_dir, 'hits')
miss_dir           = os.path.join(detections_dir, 'miss')
split_dir          = os.path.join(os.pardir, 'data_sources', subdir, 'split')

truth_log = os.path.join(detections_dir, 'truth.csv')

In [4]:
hit_files  = [f for f in os.listdir(hits_dir) if f.endswith('.jpg')]
miss_files = [f for f in os.listdir(miss_dir) if f.endswith('.jpg')]

# Initially examine the hits
global category
global image_list
global image_dir
global image_index
category   = 'hits'
image_list = hit_files
image_dir  = hits_dir
image_index = 0

## GUI

In [5]:
# Set up callback functions
def update_image():
    global category
    global image_list
    global image_dir
    global image_index
    
    if image_index >= len(image_list):
        if category == 'miss':
            return 0
        
        # Switch to processing the misses
        category = 'miss'
        image_list = miss_files
        image_dir  = miss_dir
        image_index = 0
               
    filename = os.path.join(image_dir, image_list[image_index])
    file     = open(filename, 'rb')
    image    = file.read()
    
    image_widget.value = image
    count_widget.value = '{0:d} of {1:d}'.format(image_index+1, len(image_list))
    
    return len(image_list) - image_index - 1 # Number of images to go

# Create widgets
image_widget = widgets.Image(format='jpg')
count_widget = widgets.Text(value='{0:d} of {1:d}'.format(1, len(image_list)))
hit_button   = widgets.Button(description='True Postiive')
miss_button  = widgets.Button(description='True Negative')
maybe_button = widgets.Button(description='Understandable Miss')
row          = widgets.HBox([hit_button, miss_button, maybe_button, count_widget])
out          = widgets.Output()

@out.capture()

def next_image():
    global image_index
    global image_list
    global category

    # Reset when we run out of hits, and start looking at the misses
    if category == 'hits' and image_index >= len(image_list):
        print('Making the switch!')
        image_index = 0
        catgory     = 'miss'
        image_list  = miss_files
        image_dir   = miss_dir
        
    image_index += 1
    images_remaining = update_image()

    
def on_hit(b):
    global image_index
    write_log(image_index, '1')
    next_image()
    
def on_miss(b):
    global image_index
    write_log(image_index, '0')
    next_image()

def on_maybe(b):
    global image_index
    write_log(image_index, '2')
    next_image()
    
def write_log(index, value):
    if not os.path.exists(truth_log):
        log = open(truth_log, 'w')
        log.write('filename,actual,predicted\n')
    else:
        log = open(truth_log, 'a')

    if category == 'hits':
        predicted = '1'
    else:
        predicted = '0'
    log.write('{0:s},{1:s},{2:s}\n'.format(image_list[index], value, predicted))
        
        
hit_button.on_click(on_hit)
miss_button.on_click(on_miss)
maybe_button.on_click(on_maybe)

# Start with the first image
update_image()

# Display the GUI
display(image_widget)
display(row)
display(out)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Output()